In [35]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Flatten
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os 

In [60]:
num_classes = 5
batch_size = 8
num_epochs = 2
image_size = (224,224)
# ADD DATASET FROM DIRECTORY

data_dir = r"/mnt/c/Windows/System32/repos/thesis_raw_data/Doc_H-Data/"

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                             rotation_range=20,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             validation_split=0.2,
                             )

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_ds = datagen.flow_from_directory(
  data_dir,
  classes='inferred',
  class_mode='categorical',
  subset="training",
  seed=123,
  target_size= image_size,
  batch_size= batch_size)

val_ds = val_datagen.flow_from_directory(
  data_dir,
  classes='inferred',
  class_mode='categorical',
  # validation_split=0.2,
  subset="validation",
  seed=123,
  target_size= image_size,
  batch_size= batch_size)


val_ds.samples
len(data_dir)



Found 0 images belonging to 8 classes.
Found 0 images belonging to 8 classes.


57

In [37]:
# train_ds_ub = train_ds.unbatch()
# images = list(train_ds_ub.map(lambda x, y: x))
# labels = list(train_ds_ub.map(lambda x, y: y))

# print(labels)

y = np.concatenate([y for x, y in train_ds], axis=0)
print(y)

KeyboardInterrupt: 

In [54]:
vgg16_model = VGG16(include_top = False,
            weights = 'imagenet', 
            input_tensor = None, 
            input_shape = (224,224,3), #shape of npy file data
            pooling = None,
            classes = 1000,
            classifier_activation="softmax") 


# Do not retrain convolutional layers
for layer in vgg16_model.layers:
    layer.trainable = False

input_shape = keras.Input(shape=(224, 224, 3))

# Add new fully connected layers
x = Flatten()(vgg16_model.output)
x = Dense(2048, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)  # num_classes is the number of classes in your dataset

# Create a new model with the fully connected layers added
model = Model(inputs=vgg16_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
# Use the generators to train the model
history = model.fit(train_ds,
          validation_data = val_ds,
          epochs=num_epochs,
          )
        #   steps_per_epoch= len(train_generator) // batch_size,
        #   validation_data=test_generator)


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:

# Save the trained model
model.save('vgg16_trained.h5')

# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

print(history.history['val_accuracy'])
#Find the x and y position of the highest test accuracy
#list all val_accuracy values
list = history.history['val_accuracy']

#find highest val_accuracy
ymax =  max(history.history['val_accuracy'])

#find index of highest val_accuracy
xpos = list.index(max(history.history['val_accuracy']))

# Annotation for max accuracy
plt.annotate('Max Accuracy @ {}%'.format(round(ymax*100,2)), xy=(xpos, ymax), xytext=(xpos, ymax+.05), ha = 'center', 
             arrowprops=dict(arrowstyle="->", facecolor='black'))
#Show plot             
plt.show()
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

print(history.history['val_accuracy'])
#Find the x and y position of the highest test accuracy
#list all val_accuracy values
list = history.history['val_accuracy']

#find highest val_accuracy
ymax =  max(history.history['val_accuracy'])

#find index of highest val_accuracy
xpos = list.index(max(history.history['val_accuracy']))

# Annotation for max accuracy
plt.annotate('Max Accuracy @ {}%'.format(round(ymax*100,2)), xy=(xpos, ymax), xytext=(xpos, ymax+.05), ha = 'center', 
             arrowprops=dict(arrowstyle="->", facecolor='black'))
#Show plot             
plt.show()

https://stackoverflow.com/questions/54589669/confusion-matrix-error-classification-metrics-cant-handle-a-mix-of-multilabel\
for below

In [ ]:
predicted_batch = model.predict(val_ds)
predicted_id= np.argmax(predicted_batch, axis=-1)

print(predicted_id)

true_id = tf.concat([y for x, y in val_ds], axis=0)
true_id=np.argmax(true_id, axis=1)
true_id[1]
print(true_id)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(predicted_id, true_id)
labels = ["No Damage", "Slight Damage", "Moderate Damage", "Extensive Damage", "Completely Damaged"]
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=labels)
disp.plot(cmap=plt.cm.Oranges)
plt.grid(None)
plt.show()


In [ ]:
# Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn import confusion_matrix
# true_categories = tf.concat([y for x, y in val_ds], axis=0)

# cm = confusion_matrix(true_categories, predicted_id)
# fig = plt.figure(figsize = (8,8))
# ax1 = fig.add_subplot(1,1,1)
# sns.set(font_scale=1.4) #for label size
# sns.heatmap(cm, annot=True, annot_kws={"size": 12},
#      cbar = False, cmap='Purples');
# ax1.set_ylabel('True Values',fontsize=14)
# ax1.set_xlabel('Predicted Values',fontsize=14)
# plt.show()

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

val_ds = np.argmax(val_ds, axis = -1)


predictions = model.predict(train_ds)

predictions = np.argmax(predictions)

print(val_ds)
print(predictions)
# matrix = confusion_matrix(val_ds, predictions)
# labels = ["Non-collapse", "Partial Collapse", "Global Collapse"]
# disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=labels)

# disp.plot(cmap=plt.cm.Oranges)
# plt.show()